In [10]:
import numpy as np
from datetime import date
import pandas as pd
class UserItemData:
    def __init__(self,path,start_date=None,end_date=None,min_ratings=0):
        self.start_date = start_date
        self.end_date = end_date
        self.min_ratings = min_ratings
        self.data_csv = pd.read_csv(path,sep='\t')
        if(self.start_date is not None):
            self.data_csv = self.date_from()
        if(self.end_date is not None):
            self.data_csv = self.date_to()
            
        self.data_csv = self.data_csv.pivot(index='userID', columns='movieID', values='rating')    
        self.data_csv = self.data_csv.loc[:, (self.data_csv.notnull().sum(axis=0) > self.min_ratings)]
        self.count = 0
        for i,j in self.data_csv.iterrows():
            self.count+=j.count()
        
            
    def date_from(self):
        day_s, month_s, year_s = self.start_date.split(".")
        filtered = self.data_csv[(self.data_csv['date_year'] >= int(year_s))]
        filtered = filtered.drop(filtered[(filtered['date_year'] <= int(year_s)) & (filtered['date_month'] <= int(month_s)) & (filtered['date_day'] < int(day_s))].index)
        filtered = filtered.drop(filtered[(filtered['date_year'] <= int(year_s)) & (filtered['date_month'] < int(month_s))].index)
        return filtered
    
    def date_to(self):
        day, month, year = self.end_date.split(".")
        filtered = self.data_csv[self.data_csv['date_year'] <= int(year)]
        filtered = filtered.drop(filtered[(filtered['date_year'] >= int(year)) & (filtered['date_month'] >= int(month)) & (filtered['date_day'] >= int(day))].index)
        filtered = filtered.drop(filtered[(filtered['date_year'] >= int(year)) & (filtered['date_month'] > int(month))].index)
        return filtered
    
    def nratings(self):
        return self.count

    
uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())


uim = UserItemData('data/user_ratedmovies.dat', start_date = '12.1.2007', end_date='16.2.2008', min_ratings=100)
print(uim.nratings())

855598
72784


In [11]:
#druga
import pandas as pd
class MovieData:
    def __init__(self,path):
        self.data = pd.read_csv(path,sep='\t', encoding="latin1")
     
    def get_title(self,movieID):
        for index,value in self.data.iterrows():
            if(value['id']==movieID):
                return value['title']

md = MovieData('data/movies.dat')
print(md.get_title(1))  

Toy story


In [12]:
import random

class RandomPredictor():
    def __init__(self,min_ocena,max_ocena):
        self.min_ocena = min_ocena
        self.max_ocena = max_ocena
        
    def fit(self, X):
        self.uim = X
    
    def predict(self, user_id):
        d = {}
        for movie, rat in self.uim.data_csv.loc[user_id].iteritems():
            if rat:
                rat = random.randint(self.min_ocena, self.max_ocena)         
            d[movie] = rat
        return d 
        
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item]))
    

<class 'dict'>
Film: Toy story, ocena: 2
Film: Grumpy Old Men, ocena: 3
Film: Money Train, ocena: 4
Film: The Usual Suspects, ocena: 3
Film: City Hall, ocena: 5


In [13]:
#cetvrta
#nije fixed
import operator

class Recommender:
    def __init__(self,predictor):
        self.predictor = predictor
    
    def fit(self,X):
        self.uim = X
        self.predictor.fit(X)
        
    def recommend(self,userID,n=10,rec_seen=True):
        d = self.predictor.predict(userID)
        if rec_seen == False:
            seznam = [i for i in self.uim.data_csv.loc[userID][self.uim.data_csv.loc[userID].notnull()].index]
            for i in seznam:
                if i in d.keys():
                    del d[i]
        
        dic = dict(sorted(d.items(), key=operator.itemgetter(1),reverse=True))
        final = []
        for index,key in enumerate(dic.keys()):
            if(index < n):
                final.append((key,dic[key]))
        return final
            
        
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')

rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Film: Waiting to Exhale, ocena: 5
Film: Nixon, ocena: 5
Film: Get Shorty, ocena: 5
Film: Copycat, ocena: 5
Film: Now and Then, ocena: 5


In [5]:
class AveragePredictor():
    def __init__(self,b=0):
        if(b < 0):
            self.b = 0
        else:
            self.b = b
    
    def fit(self, X):
        self.uim = X
        self.d = {}
        g_avg = self.uim.data_csv.sum().sum() / self.uim.count
        #avg = (vs + b * g_avg) / (n + b)
        for i in self.uim.data_csv:
            vs = self.uim.data_csv[i].sum()
            n = self.uim.data_csv[i].count()
            avg = (vs + self.b * g_avg) / (n+self.b)
            self.d[i] = avg
            
    
    def predict(self,userID):
        return self.d
        
            
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')

rp = AveragePredictor(b=100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))        
    

Film: The Usual Suspects, ocena: 4.225944245560473
Film: The Godfather: Part II, ocena: 4.146907937910189
Film: Cidade de Deus, ocena: 4.116538340205236
Film: The Dark Knight, ocena: 4.10413904093503
Film: 12 Angry Men, ocena: 4.103639627096175


In [6]:
class ViewsPredictor():
    def fit(self, X):
        self.uim = X
    def predict(self, userID):
        d = {}
        for i in self.uim.data_csv:
            d[i] = self.uim.data_csv[i][self.uim.data_csv[i].notnull()].count()
        return d

        
        
rp = ViewsPredictor()
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) 

Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film: The Lord of the Rings: The Two Towers, ocena: 1528
Film: The Lord of the Rings: The Return of the King, ocena: 1457
Film: The Silence of the Lambs, ocena: 1431
Film: Shrek, ocena: 1404


In [7]:
class STDPredictor:
    def __init__(self, min_ratings):
        self.min_ratings = min_ratings
    
    def fit(self, X):
        self.uim = X
        
    def predict(self, userID):
        d = {}
        for i in uim.data_csv:
            if(uim.data_csv[i][uim.data_csv[i].notnull()].count() >= self.min_ratings):
                d[i] = np.std(uim.data_csv[i])
        return d


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = STDPredictor(100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))         


Film: Plan 9 from Outer Space, ocena: 1.3386525190884058
Film: The Passion of the Christ, ocena: 1.2790640858537328
Film: The Texas Chainsaw Massacre, ocena: 1.2306963889088796
Film: Jackass Number Two, ocena: 1.2134235427973517
Film: White Chicks, ocena: 1.1841675574946435


In [14]:
from itertools import combinations
class ItemBasedPredictor:
    def __init__(self,min_values=0,threshold=0):
        self.min_values = 0
        self.threshold = 0
    def fit(self,X):
        self.uim = X
        self.all_sims = {}
        
        movies = [i for i in self.uim.data_csv]
        movies_comb = list(combinations(movies,2))
        
        for prvi, drugi in movies_comb:
            if not (drugi,prvi) in self.all_sims.keys():
                sim = self.similarity(prvi,drugi)
                self.all_sims[(prvi,drugi)] = sim
    
        
    def similarity(self, p1, p2):
        filtered = self.uim.data_csv[(self.uim.data_csv[p1].notnull()) & (self.uim.data_csv[p2].notnull())]
        p1_rat = filtered[p1]
        p2_rat = filtered[p2]
        mean_rat = np.array(filtered.mean(axis=1))
        
        gore = sum((p1_rat - mean_rat) * (p2_rat - mean_rat))
        prvi_sqr = sum((p1_rat - mean_rat) ** 2)
        drugi_sqr = sum((p2_rat - mean_rat)**2)
        dole = np.sqrt(prvi_sqr) * np.sqrt(drugi_sqr)
        
        st_rat = len(matrix.index)
        rez = gore/dole
        
        if(rez < 0  or st_rat < self.min_values):
            return 0
        
        return gore/dole
    
    def predict(self, userID):
        d = {}
        #all not rated movies
        movies = self.uim.data_csv.loc[userID][self.uim.data_csv.loc[userID].isnull()].index.tolist()
        #(sim izmadju dva filma * ocena tog usera za taj film drugi )
        for movie in movies:
            gore = 0
            dole = 0
            for index, ratings in self.uim.data_csv.iteritems():
                if(index != movie and index not in movies):
                    ocena = self.uim.data_csv.loc[userID][index]
                    par = tuple(sorted([movie,index]))
                    sim = self.all_sims[par]
                    gore += sim * ocena
                    dole += sim
            if(movie not in d.keys() and dole > 0):
                d[movie] = gore/dole
        return d
    
    def num_of_similar(self, number):
        dic = dict(sorted(self.all_sims.items(), key=operator.itemgetter(1),reverse=True))    
        final = []
        for index,key in enumerate(dic.keys()):
            if(index < number):
                final.append((key,dic[key]))
        return final
    
    def similarItems(self, item, n):
        most_similar = {}
        for par in self.all_sims.keys():
            if(item in par):
                if(par.index(item) == 0):
                    most_similar[par[1]] = self.all_sims[par]
                else:
                    most_similar[par[0]] = self.all_sims[par]
                    
        dic = dict(sorted(most_similar.items(), key=operator.itemgetter(1),reverse=True)) 
        final = []
        for index,key in enumerate(dic.keys()):
            if(index < n):
                final.append((key,dic[key]))
        return final
                
        
uim = UserItemData('data/user_ratedmovies.dat')
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
#print(uim.movies)
print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))


MemoryError: 

In [ ]:
print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

In [ ]:
top20 = rp.num_of_similar(20)
for (i, j), val in top20:
    print("Film1: {}, Film2: {}, podobnost: {}".format(md.get_title(i), md.get_title(j), val))

In [ ]:
rec_items = rp.similarItems(4993, 20)
print('Filmi podobni "The Lord of the Rings: The Fellowship of the Ring": ')
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))